In [5]:
import numpy as np
import matplotlib.pyplot as plt
from pykrige.uk import UniversalKriging
import gstools as gs

s=9

#Campo aleatorio en 2D
np.random.seed(s)
L = 10
grid_size = 50
X = np.linspace(0,L,grid_size)
Y = np.linspace(0,L,grid_size)
XX, YY = np.meshgrid(X,Y)
lim = [0,L,0,L]
campo_real = 5*np.sin(0.5*XX)+2*np.cos(0.5*YY)+np.random.normal(0,0.05,XX.shape)

#Campo aletorio con gstools
model = gs.Gaussian(dim=2,var=10,len_scale=5)
srf = gs.SRF(model,seed=s)
campo_real = srf((X,Y),mesh_type='structured')

#Puntos iniciales
n_puntos = 10
puntos_x = np.random.rand(n_puntos)*L
puntos_y = np.random.rand(n_puntos)*L
puntos_z = [srf([puntos_x[i],puntos_y[i]],seed=s) for i in range(n_puntos)]
#puntos_z = np.array(puntos_z) ?

def variogram(param,h):
    var = param[0] 
    q = param[1]
    theta = param[2]
    cor = np.exp(-(abs(h)**q)/theta)
    return (var**2)*(1-cor)
#No sé como meter anisotropía en la función, ya que UniversalKriging toma el valor de h
#Hay que definir el variograma, no la covarianza
#No satura la varianza a (var**2)?

theta = 50
q = 2
sigma = 0.5
uk = UniversalKriging(
    puntos_x,puntos_y,puntos_z,
    variogram_model="custom",variogram_parameters=[theta,q,sigma],variogram_function=variogram,
    drift_terms=["regional_linear"]
)
campo_estimado,varianza_estimada = uk.execute("grid",X,Y)


#Graficar los resultados
fig,axes = plt.subplots(1,3,figsize=(15,5))

#Al ponerlo en los ejes
#Campo Real
ax = axes[0]
im = ax.imshow(np.transpose(campo_real),extent=lim,origin="lower")
ax.scatter(puntos_x,puntos_y,color="white",marker="o",label="Puntos de medición")
ax.set_title("Campo Real")
plt.colorbar(im)
ax.legend()

#Campo estimado
ax = axes[1]
im = ax.imshow(campo_estimado,extent=lim,origin="lower")
ax.scatter(puntos_x,puntos_y,color="white",marker="o",label="Puntos de medición")
ax.set_title("Campo Estimado")
plt.colorbar(im)
ax.legend()

#Varianza estimada
ax = axes[2]
im = ax.imshow(varianza_estimada,extent=lim,origin="lower")
ax.scatter(puntos_x,puntos_y,color="white",marker="o",label="Puntos de medición")
ax.set_title("Varianza Estimada")
plt.colorbar(im)
ax.legend()

plt.tight_layout()
plt.show()

#Comprobación de puntos
var_puntos = []
campo_puntos = []
for i in range(n_puntos):
    var_punto = uk.execute("points",[puntos_x[i]],[puntos_y[i]])[1][0]
    var_puntos.append(var_punto)
    campo_punto = uk.execute("points",[puntos_x[i]],[puntos_y[i]])[0][0]
    campo_puntos.append(campo_punto)

print('Campo real')
[print('El valor del campo en el punto ',[puntos_x[i],puntos_y[i]],' es ',puntos_z[i]) for i in range(n_puntos)]
print()

print('Campo estimado')
[print('El valor del campo en el punto ',[puntos_x[i],puntos_y[i]],' es ',campo_puntos[i]) for i in range(n_puntos)]
print(f"Campo mínimo: {np.min(campo_estimado)}, máximo: {np.max(campo_estimado)}")
print()

print('Varianza estimada')
[print('El valor del campo en el punto ',[puntos_x[i],puntos_y[i]],' es ',var_puntos[i]) for i in range(n_puntos)]
print(f"Varianza mínima: {np.min(varianza_estimada)}, máxima: {np.max(varianza_estimada)}")

Campo real
El valor del campo en el punto  [8.988699030920687, 0.8046120889417718]  es  [-3.84982282]
El valor del campo en el punto  [6.676116433222271, 5.692931522156689]  es  [-2.55047898]
El valor del campo en el punto  [0.7003583370788036, 5.139768405395309]  es  [4.93563581]
El valor del campo en el punto  [2.649007439195614, 2.032010180724062]  es  [0.63334142]
El valor del campo en el punto  [4.9886595143520305, 4.188561782361089]  es  [-0.46851165]
El valor del campo en el punto  [8.956605560291187, 2.8518177583413094]  es  [-4.18253872]
El valor del campo en el punto  [5.864503205671924, 0.2931413899508828]  es  [-0.8767204]
El valor del campo en el punto  [1.361586745836102, 5.7964649603716945]  es  [5.81264251]
El valor del campo en el punto  [4.274813341510625, 6.917383677026514]  es  [2.63740097]
El valor del campo en el punto  [2.3008308251747156, 8.948261609713843]  es  [7.93014961]

Campo estimado
El valor del campo en el punto  [8.988699030920687, 0.8046120889417718] 

In [6]:
#Upper bound confidence

from scipy.optimize import minimize

np.random.seed(s)
#Probar por cuadratura Gauss-Hermite?
#Mejor con interpolación que con punto de la malla?
def ubc_fun(pos):
    phi = uk.execute("points",[pos[0]],[pos[1]])[0][0]
    var = uk.execute("points",[pos[0]],[pos[1]])[1][0]
    return -(phi-b*np.sqrt(abs(var)))

lista_x = puntos_x.tolist()
lista_y = puntos_y.tolist()
lista_z = puntos_z.copy()
b = 0.001
max_est = minimize(ubc_fun,[lista_x[0],lista_y[0]],bounds=[(0,L),(0,L)],method='L-BFGS-B')
new_x,new_y = max_est.x
max_ubc = -max_est.fun
lista_x.append(new_x)
lista_y.append(new_y)
lista_z.append(srf([new_x,new_y],seed=s))

print('lista_x = ',lista_x)
print('lista_y = ',lista_y)
print('lista_z = ',lista_z)

campo_antes = campo_estimado.copy()
var_antes = varianza_estimada.copy()

#Graficar los resultados
fig,axes = plt.subplots(2,3,figsize=(15,8))

ax = axes[0,0]
im = ax.imshow(np.transpose(campo_real),extent=lim,origin="lower")
ax.scatter(lista_x,lista_y,color="white",marker="o",label="Puntos de medición")
ax.set_title("Campo Real")
plt.colorbar(im)
ax.legend()

ax = axes[1,0]
im = ax.imshow(campo_antes-b*np.sqrt(abs(var_antes)),extent=lim,origin="lower")
ax.scatter(lista_x,lista_y,color="white",marker="o",label="Puntos de medición")
ax.set_title("UBC")
plt.colorbar(im)
ax.legend()

ax = axes[0,1]
im = ax.imshow(campo_antes,extent=lim,origin="lower")
ax.scatter(lista_x,lista_y,color="white",marker="o",label="Puntos de medición")
ax.set_title("Campo Estimado Antes")
plt.colorbar(im)
ax.legend()

ax = axes[1,1]
im = ax.imshow(var_antes,extent=lim,origin="lower")
ax.scatter(lista_x,lista_y,color="white",marker="o",label="Puntos de medición")
ax.set_title("Varianza Estimada Antes")
plt.colorbar(im)
ax.legend()

uk = UniversalKriging(
    lista_x,lista_y,lista_z,
    variogram_model="custom",variogram_parameters=[theta,q,sigma],variogram_function=variogram,
    drift_terms=["regional_linear"]
)
campo_est,varianza_est = uk.execute("grid",X,Y)

ax = axes[0,2]
im = ax.imshow(campo_est,extent=lim,origin="lower")
ax.scatter(lista_x,lista_y,color="white",marker="o",label="Puntos de medición")
ax.set_title("Campo Estimado Después")
plt.colorbar(im)
ax.legend()

ax = axes[1,2]
im = ax.imshow(varianza_est,extent=lim,origin="lower")
ax.scatter(lista_x,lista_y,color="white",marker="o",label="Puntos de medición")
ax.set_title("Varianza Estimada Después")
plt.colorbar(im)
ax.legend()

plt.tight_layout()
plt.show()

lista_x =  [8.988699030920687, 6.676116433222271, 0.7003583370788036, 2.649007439195614, 4.9886595143520305, 8.956605560291187, 5.864503205671924, 1.361586745836102, 4.274813341510625, 2.3008308251747156, 0.0]
lista_y =  [0.8046120889417718, 5.692931522156689, 5.139768405395309, 2.032010180724062, 4.188561782361089, 2.8518177583413094, 0.2931413899508828, 5.7964649603716945, 6.917383677026514, 8.948261609713843, 10.0]
lista_z =  [array([-3.84982282]), array([-2.55047898]), array([4.93563581]), array([0.63334142]), array([-0.46851165]), array([-4.18253872]), array([-0.8767204]), array([5.81264251]), array([2.63740097]), array([7.93014961]), array([5.96329298])]
